In [1]:
import numpy as np 
import pandas as pd  

import requests
from bs4 import BeautifulSoup

print('Libraries imported')

Libraries imported


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).text
soup = BeautifulSoup(results,'html.parser')

In [3]:
# I found that the table what I want is from 1st td to 867th td 
toronto_FSAs = soup.find_all('td')[0:867]
toronto_FSAs[:10]

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>]

In [4]:
postcode_lsts= list(range(0,867,3))
borough_lsts= list(range(1,867,3))
neigh_lsts= list(range(2,867,3))
postcode = []
borough = [] 
neighbourhood = [] 

for pscode in postcode_lsts:
    postcode.append(toronto_FSAs[pscode].text)
    
for bor in borough_lsts:
    borough.append(toronto_FSAs[bor].text)

for neigh in neigh_lsts:
    neighbourhood.append(toronto_FSAs[neigh].text.replace('\n',''))



df_toronto_postcode = pd.DataFrame({
    'Postcode':postcode,
    'Borough':borough,
    'Neighbourhood':neighbourhood
})
df_toronto_postcode.head(12)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
df_toronto_postcode.loc[df_toronto_postcode['Neighbourhood'] == 'Not assigned','Neighbourhood'] = "Queen's Park"

In [6]:
# remove 'Not assigned'
df_toronto_postcode = df_toronto_postcode.loc[df_toronto_postcode['Borough'] != 'Not assigned',]
df_toronto_postcode.head(11)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [7]:
# remove duplicated 
df_toronto_postcode_new = df_toronto_postcode.drop_duplicates(['Postcode'], keep='first')
df_toronto_postcode_new.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
14,M3B,North York,Don Mills North
15,M4B,East York,Woodbine Gardens
17,M5B,Downtown Toronto,Ryerson


In [8]:
df_toronto_postcode_new.shape

(103, 3)

In [9]:
psc_lsts = df_toronto_postcode.Postcode.unique()
neigh_lsts = []

for psc in psc_lsts:
    lsts = list(df_toronto_postcode.loc[df_toronto_postcode['Postcode'] == psc,'Neighbourhood'])
    neigh_lsts.append(', '.join(lsts))
    
df_toronto_postcode_new.Neighbourhood = neigh_lsts


C:\Users\minda\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
df_toronto_postcode_new = df_toronto_postcode_new.reset_index(drop = True)
df_toronto_postcode_new.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [20]:
df2 = pd.read_csv('./Data/Geospatial_Coordinates.csv')
df2.columns = ['Postcode','Latitude','Longitude']
df_with_LL = pd.merge(df_toronto_postcode_new,df2)

In [23]:
df_with_LL.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
